In [1]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz

--2020-05-12 08:16:40--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 495854086 (473M) [application/x-gzip]
Saving to: ‘reviews_Electronics_5.json.gz’

reviews_Electronics 100%[===================>] 472.88M  7.73MB/s    in 42s     

2020-05-12 08:17:22 (11.3 MB/s) - ‘reviews_Electronics_5.json.gz’ saved [495854086/495854086]



In [0]:
import gzip
import re
from nltk.corpus import stopwords

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

In [0]:
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def clean_line(line):
    line = simple_preprocess(str(line), deacc=True)
    filtered_content = []
    for word in line:
        if word not in stop_words and word.isalpha():
            filtered_content.append(word)
    return filtered_content

In [0]:
def read_dataset(fname):
    count = 0
    exp_dataset = []
    for review in parse(fname):
        line = review["reviewText"]
        new_line = clean_line(line)
        exp_dataset.append(new_line)
        count += 1
        if count > 10000:
            break
    return exp_dataset

In [0]:
r = read_dataset("reviews_Electronics_5.json.gz")


In [0]:
import gensim

In [13]:
bigram = gensim.models.Phrases(r, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[r], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [15]:
data_words_bigrams = make_bigrams(r)
data_words_trigrams = make_trigrams(r)



import spacy
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

# Create Dictionary
import gensim.corpora as corpora
dictionary = corpora.Dictionary(data_lemmatized)

# Create Corpus
corpus = [dictionary.doc2bow(text) for text in data_lemmatized]

[['get', 'trucker', 'impress', 'shipping', 'time', 'arrive', 'day', 'earlier', 'expect', 'week', 'use', 'however', 'start', 'freeze', 'could', 'glitch', 'unit', 'work', 'great', 'worked', 'work', 'great', 'normal', 'person', 'trucker', 'option', 'big', 'truck', 'route', 'tell', 'scale', 'come', 'ect', 'love', 'big', 'screen', 'ease', 'use', 'ease', 'putting', 'address', 'memory', 'really', 'bad', 'say', 'unit', 'exception', 'freeze', 'probably', 'luck', 'contact', 'seller', 'minute', 'email', 'receive', 'email', 'instruction', 'exchange', 'impress', 'way', 'around']]


In [0]:
from gensim.models import Word2Vec
import multiprocessing
import time

In [18]:
w2v_model=gensim.models.Word2Vec(data_lemmatized,size=300,min_count=40,window=10,max_vocab_size=10000,negative=10)
w2v_model.train(data_lemmatized,epochs=30,total_examples=w2v_model.corpus_count)

(10279691, 13726800)

In [20]:
w2v_model.doesnt_match('run walk jump swim'.split())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'walk'

In [21]:
w2v_model.doesnt_match('laptop computer processor swim'.split())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'computer'